# Stock Data Scraper Function

In [1]:
import pandas as pd
import os
#%pip install pytrends
import pytrends
from pytrends.request import TrendReq
#%pip install pageviewapi
import pageviewapi
#%pip install yfinance
import yfinance as yf

In [160]:
def Big_scraper(kw_list_1, kw_list_2, ticker, start,end):
    
    """
    Description:
    ------------
    
    The function initially grabs historical, indexed, hourly data for when the keyword 
    was searched most as shown on Google Trends' Interest Over Time section.
    It then cleans the data to show daily hits on the keyword in Google news.

    
    input:
    -----
    kw_list_1: List of up to 5 key words that will be scraped from google trends for the dates given.
             Here, the scraping will pull the total posted items in google news that contains
             one of the key words.
    
    kw_list_2: List of wikipedia article titles (unlimited length) that will pull the amount of
            views the article recieved each day. 

    ticker: the ticker abriviation of the desired stock. Must be netered in as an all capitalized string 
    example Apple Inc. woud be "AAPL"
             
    start: the start of the desired timeline you want scrape. Date Must be entered in as "YYYYMMDD"
    
    end: the end of the desired timeline you want scrape. Date Must be entered in as "YYYYMMDD"
             
    return:
    -------
    
    combined: a dataframe containing the sum of the daily keyword hits in google news (key words labeled _x),
    
    data frame cointaing stock info including open, close, high, low prices of the stock,
    as well as the stocks daily trading volume and the amount if there was a split or dividend 
    preformed on the stock that day,
    
    and the sum of how many times key word wikipedia pages were viewed in a day (key words labeled _y)
    """
    
    year_s = int(start[0:4])
    month_s = int(start[4:6])
    day_s = int(start[6:8])
    year_e = int(end[0:4])
    month_e = int(end[4:6])
    day_e = int(end[6:8])
    
    starter = pd.to_datetime(f"'{year_s}-{month_s}-{day_s}'")
    ender = pd.to_datetime(f"'{year_e}-{month_e}-{day_e}'")
    
    
    pytrends = TrendReq(hl='en-US', tz=360, retries=10)
    jeff = pytrends.get_historical_interest(kw_list_1, \
                                 year_start = year_s, month_start = month_s, day_start = day_s, hour_start = 1, \
                                 year_end = year_e, month_end = month_e, day_end = day_e, hour_end = 23, \
                                 cat = 0, geo = '', gprop = 'news', sleep = 60)
    
    jeff = jeff.iloc[:, 0:-1] # eliminates the isPartial Column
    jeff = jeff.reset_index().drop_duplicates(subset = "date") #removing duplicates from the index
    jeff = jeff.groupby(pd.Grouper(key="date", freq="D")).mean() # coverts to the mean of daily scores

    dow = yf.Ticker("^DJI")
    dow_h = dow.history(start=starter, end=ender)
    dow_h = pd.DataFrame(dow_h)
    dow_names = {"Open":"dow_open","Close":"dow_close","Low": "dow_low",
    'High':'dow_high','Volume':'dow_vol'}
    dow_h=dow_h.rename(dow_names, axis=1).drop(["Dividends","Stock Splits"], axis=1)

    nas = yf.Ticker("^IXIC")
    nas_h = nas.history(start=starter, end=ender)
    nas_h = pd.DataFrame(nas_h)
    nas_names = {"Open":"nas_open", "Close":"nas_close", "Low": "nas_low",
    'High':'nas_high','Volume':'nas_vol'}
    nas_h=nas_h.rename(nas_names, axis=1).drop(["Dividends","Stock Splits"], axis=1)

    market = dow_h.merge(nas_h,left_index=True, right_index=True, how="left")
    
    tick = yf.Ticker(ticker)
    hist = tick.history(start=starter, end=ender)
    hist = pd.DataFrame(hist)
    
    combined = jeff.merge(hist, left_index=True, right_index=True, how="left")
    combined = combined.merge(market, left_index=True, right_index=True, how="left")  
    
    d = pd.DataFrame()
    for key_word in kw_list_2:
        geoff = pageviewapi.per_article('en.wikipedia', key_word, start, end,
                                    access='all-access', agent='all-agents', granularity='daily')
        dicty = dict(geoff)
        views = pd.DataFrame(dicty["items"])
        views["timestamp"] = pd.to_datetime((views["timestamp"]), format="%Y%m%d%H")
        views = views.set_index("timestamp")
        page = pd.Series(views["views"])
        d[key_word] = page
        
    combined = combined.merge(d, left_index=True, right_index=True, how="left") 
    
    return combined

In [168]:
#example pull
kw_list_1 = ["Apple", "Apple Inc.", "IPhone", "MacBook", "MacOS"]
kw_list_2 = ["Apple Inc.", "IPhone"]
Big_scraper(kw_list_1,kw_list_2,"AAPL","20211201","20211214")

,Apple,Apple Inc._x,IPhone_x,MacBook,MacOS,Open,High,Low,Close,Volume,...,dow_low,dow_close,dow_vol,nas_open,nas_high,nas_low,nas_close,nas_vol,Apple Inc._y,IPhone_y
date,,,,,,,,,,,,,,,,,,,,,
2021-12-01,18.217391,0.0,19.782609,2.086957,0.260870,167.266892,170.083311,164.320648,164.560349,152052500.0,...,34006.980469,34022.039062,496000000.0,15752.269531,15816.820312,15243.929688,15254.049805,6.266020e+09,14391,8014
2021-12-02,22.250000,0.0,25.583333,1.166667,0.000000,158.538019,163.991063,157.599213,163.551620,136739200.0,...,34076.250000,34639.789062,466900000.0,15181.820312,15444.540039,15150.120117,15381.320312,5.390100e+09,14364,8072
2021-12-03,15.791667,0.0,16.916667,2.708333,0.250000,163.811298,164.750104,159.516766,161.634064,118023100.0,...,34264.570312,34580.078125,439550000.0,15428.709961,15470.360352,14931.059570,15085.469727,5.859520e+09,13359,7546
2021-12-04,15.833333,0.0,24.291667,1.375000,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11606,7311
2021-12-05,16.791667,0.0,21.083333,1.041667,0.291667,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12245,7586
2021-12-06,14.500000,0.0,18.791667,4.416667,0.000000,164.080946,167.666390,164.070964,165.109650,107497000.0,...,34633.429688,35227.031250,416720000.0,15117.629883,15281.990234,14931.610352,15225.150391,5.095960e+09,13862,8381
2021-12-07,16.750000,0.0,18.083333,2.666667,0.000000,168.864855,171.361674,168.125791,170.962173,120405400.0,...,35423.988281,35719.429688,474940000.0,15510.910156,15720.089844,15507.660156,15686.919922,5.091220e+09,14563,8188
2021-12-08,15.291667,0.0,23.625000,2.708333,0.000000,171.910980,175.736109,170.482792,174.857224,116998900.0,...,35602.648438,35754.750000,387650000.0,15690.650391,15792.639648,15618.879883,15786.990234,4.600800e+09,14027,7697
2021-12-09,17.875000,0.0,27.583333,2.375000,0.000000,174.687436,176.525091,173.698690,174.337875,108923700.0,...,35577.140625,35754.691406,353020000.0,15720.540039,15796.049805,15511.120117,15517.370117,4.484230e+09,13975,7590


## Individual Functions (with examples)

In [16]:
def google_trends(kw_list, start, end):
    
    """
    Description:
    ------------
    
    The function initially grabs historical, indexed, hourly data for when the keyword 
    was searched most as shown on Google Trends' Interest Over Time section.
    It then cleans the data to show daily hits on the keyword in Google news.

    
    input:
    -----
    kw_list: List of up to 5 key words that will be scraped from the timeline given to the function.
             Here, the scraping will pull the total posted items in google news. the contains
             one of the key words.
             
    start: the start of the desired timeline you want scrape. Date Must be entered in as "YYYYMMDD"
    
    end: Self explanetory, Date Must be entered in as "YYYYMMDD"
    
             
    return:
    -------
    
    jeff: a dataframe containing the mean of the daily google trends index score for up to 5 keywords
    """
    year_s = int(start[0:4])
    month_s = int(start[4:6])
    day_s = int(start[6:8])
    year_e = int(end[0:4])
    month_e = int(end[4:6])
    day_e = int(end[6:8])
      
    pytrends = TrendReq(hl='en-US', tz=360, retries=10)
    jeff = pytrends.get_historical_interest(kw_list, \
                                 year_start = year_s, month_start = month_s, day_start = day_s, hour_start = 1, \
                                 year_end = year_e, month_end = month_e, day_end = day_e, hour_end = 23, \
                                 cat = 0, geo = '', gprop = 'news', sleep = 60)
    
    jeff = jeff.iloc[:, 0:-1] # eliminates the isPartial Column
    jeff = jeff.reset_index().drop_duplicates(subset = "date") #removing duplicates from the index
    jeff = jeff.groupby(pd.Grouper(key="date", freq="D")).sum() # coverts to the sum of daily posts
    
    return jeff

In [22]:
jeff1= google_trends(kw_list_1, "20210101","20210114")
jeff1.head()

,Apple,Apple Inc.,IPhone,MacBook,MacOS
date,,,,,
2021-01-01,347,0,518,26,0
2021-01-02,451,0,433,45,0
2021-01-03,412,0,467,25,0
2021-01-04,424,0,382,49,0
2021-01-05,456,0,445,32,0


In [19]:
def stock_stats(ticker: str, start: str, end: str):
    """
    Description: Scrapes historial daily stock data from the Yahoo Fince sight
    and returns a dataframe containing daily open, close, high, low prices of the stock,
    as well as the stocks daily trading volume and the amount if there was a split or dividend 
    preformed on the stock that day.
    
    inputs:
    ------
    ticker: the ticker abriviation of the desired stock. Must be netered in as an all capitalized string 
    example Apple Inc. woud be "AAPL"
    
    start: the start of the desired timeline you want scrape. Date Must be entered in as "YYYYMMDD"
    
    end: Self explanetory, Date Must be entered in as "YYYYMMDD"
    
    return:
    ------
    hist: dataframe containing open, close, high, low prices of the stock,
    as well as the stocks daily trading volume and the amount if there was a split or dividend 
    preformed on the stock that day. 
    
    As well as the open close, high, low, volume, of the NASDAQ and DOW Jones Indudtiral Average
    
    """
    year_s = int(start[0:4])
    month_s = int(start[4:6])
    day_s = int(start[6:8])
    year_e = int(end[0:4])
    month_e = int(end[4:6])
    day_e = int(end[6:8])
    
    starter = pd.to_datetime(f"'{year_s}-{month_s}-{day_s}'")
    ender = pd.to_datetime(f"'{year_e}-{month_e}-{day_e}'")
    
    dow = yf.Ticker("^DJI")
    dow_h = dow.history(start=starter, end=ender)
    dow_h = pd.DataFrame(dow_h)
    dow_names = {"Open":"dow_open","Close":"dow_close","Low": "dow_low",
    'High':'dow_high','Volume':'dow_vol'}
    dow_h=dow_h.rename(dow_names, axis=1).drop(["Dividends","Stock Splits"], axis=1)

    nas = yf.Ticker("^IXIC")
    nas_h = nas.history(start = starter, end = ender)
    nas_h = pd.DataFrame(nas_h)
    nas_names = {"Open":"nas_open", "Close":"nas_close", "Low": "nas_low",
    'High':'nas_high','Volume':'nas_vol'}
    nas_h=nas_h.rename(nas_names, axis=1).drop(["Dividends","Stock Splits"], axis=1)

    market = dow_h.merge(nas_h,left_index=True, right_index=True, how="left")


    tick = yf.Ticker(ticker)
    hist = tick.history(start=starter, end=ender)
    hist = pd.DataFrame(hist)

    hist = hist.merge(market,left_index=True, right_index=True, how="left")
    
    return hist

In [21]:
jeff2= stock_stats("AAPL", "20210101","20210114")
jeff2.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,dow_open,dow_high,dow_low,dow_close,dow_vol,nas_open,nas_high,nas_low,nas_close,nas_vol
Date,,,,,,,,,,,,,,,,,
2021-01-04,132.533082,132.622413,125.823047,128.453461,143301900,0,0,30627.470703,30674.279297,29881.820312,30223.890625,475080000,12958.519531,12958.719727,12543.240234,12698.450195,6546740000
2021-01-05,127.937286,130.766226,127.480679,130.041611,97664900,0,0,30204.250000,30504.890625,30141.779297,30391.599609,350910000,12665.650391,12828.269531,12665.650391,12818.959961,6904420000
2021-01-06,126.775939,130.081327,125.445840,125.664215,155088000,0,0,30362.779297,31022.650391,30313.070312,30829.400391,500430000,12666.150391,12909.629883,12649.990234,12740.790039,7648340000
2021-01-07,127.411196,130.657029,126.914892,129.952271,109578200,0,0,30901.179688,31193.400391,30897.859375,31041.130859,427810000,12867.339844,13090.910156,12867.339844,13067.480469,6777010000
2021-01-08,131.451110,131.649643,129.267374,131.073929,105158200,0,0,31069.580078,31140.669922,30793.269531,31097.970703,381150000,13160.219727,13208.089844,13036.549805,13201.980469,7223660000


In [23]:
def wiki_scraper(kw_list: list, start_date: str, end_date: str):
    '''
    Description: Pulls the sum of how many times a wikipedia page was viewed that day
    
    inputs:
    ------
    
    kw_list: list of wikipedia page names to be scrpapped, can be of unlimted length
    
    start_date: the start of the desired timeline you want scrape. Date Must be entered in as "YYYYMMDD"
    
    end_date: the end of the desired timeline you want scrape. Date Must be entered in as "YYYYMMDD"
    
    '''
    d = pd.DataFrame()
    for key_word in kw_list:
        geoff = pageviewapi.per_article('en.wikipedia', key_word, start_date, end_date,
                                    access='all-access', agent='all-agents', granularity='daily')
        dicty = dict(geoff)
        views = pd.DataFrame(dicty["items"])
        views["timestamp"] = pd.to_datetime((views["timestamp"]), format="%Y%m%d%H")
        views = views.set_index("timestamp")
        page = pd.Series(views["views"])
        d[key_word] = page
        
    return d

In [26]:
jeff3 = wiki_scraper(kw_list, '20211201', '20211231')
jeff3.head()

,Apple,Apple Inc.,IPhone,MacBook,MacOS
timestamp,,,,,
2021-12-01,8237,14391,8014,704,133006
2021-12-02,8299,14364,8072,619,42741
2021-12-03,8040,13359,7546,559,37979
2021-12-04,7673,11606,7311,604,26680
2021-12-05,7810,12245,7586,588,26376


In [28]:
def joiner(google_trends, yahoo_finace, wiki_pagecount):
    """
    Description: joins all stock data sets into one dataframe, after they have been cleaned and variables added
    
    input:
    ------
    google_trends: data frame counting daily hit counts for google news stories on specific key words
    
    yahoo_finace: data frame cointaing stock info including open, close, high, low prices of the stock,
    as well as the stocks daily trading volume and the amount if there was a split or dividend 
    preformed on the stock that day.
    
    wiki_pagecount: the sum of how many times key wikipedia pages were viewed in a day
    """
    
    combined = google_trends.merge(yahoo_finace, left_index=True, right_index=True, how="left")
    combined = combined.merge(wiki_pagecount, left_index=True, right_index=True, how="left")
    return combined

### Data Scraper Without Google Trends

In [130]:
def Market_scraper(kw_list_2, ticker, start,end):
    
    """
    Description:
    ------------
    
    The function initially grabs historical, indexed, hourly data for when the keyword 
    was searched most as shown on Google Trends' Interest Over Time section.
    It then cleans the data to show daily hits on the keyword in Google news.

    
    input:
    -----
    ### kw_list_1: List of up to 5 key words that will be scraped from google trends for the dates given.
             Here, the scraping will pull the total posted items in google news that contains
             one of the key words ### removed to be edited and replaced later .
    
    kw_list_2: List of wikipedia article titles (unlimited length) that will pull the amount of
            views the article recieved each day. 

    ticker: the ticker abriviation of the desired stock. Must be netered in as an all capitalized string 
    example Apple Inc. woud be "AAPL"
             
    start: the start of the desired timeline you want scrape. Date Must be entered in as "YYYYMMDD"
    
    end: the end of the desired timeline you want scrape. Date Must be entered in as "YYYYMMDD"
             
    return:
    -------
    
    ### combined: a dataframe containing the sum of the daily keyword hits in google news (key words labeled _x) ### to be edited,
    
    the individual stock info including open, close, high, low prices of the stock,
    as well as the stocks daily trading volume and the amount if there was a split or dividend 
    preformed on the stock that day,

    as well as the open close, high, low, volume, of the NASDAQ and DOW Jones Indudtiral Average
    
    and the sum of how many times key word wikipedia pages were viewed in a day (key words labeled _y)
    """
    
    year_s = int(start[0:4])
    month_s = int(start[4:6])
    day_s = int(start[6:8])
    year_e = int(end[0:4])
    month_e = int(end[4:6])
    day_e = int(end[6:8])
    
    starter = pd.to_datetime(f"'{year_s}-{month_s}-{day_s}'")
    ender = pd.to_datetime(f"'{year_e}-{month_e}-{day_e}'")
    
    dow = yf.Ticker("^DJI")
    dow_h = dow.history(start=starter, end=ender)
    dow_h = pd.DataFrame(dow_h)
    dow_names = {"Open":"dow_open","Close":"dow_close","Low": "dow_low",
    'High':'dow_high','Volume':'dow_vol'}
    dow_h=dow_h.rename(dow_names, axis=1).drop(["Dividends","Stock Splits"], axis=1)

    nas = yf.Ticker("^IXIC")
    nas_h = nas.history(start=starter, end=ender)
    nas_h = pd.DataFrame(nas_h)
    nas_names = {"Open":"nas_open", "Close":"nas_close", "Low": "nas_low",
    'High':'nas_high','Volume':'nas_vol'}
    nas_h=nas_h.rename(nas_names, axis=1).drop(["Dividends","Stock Splits"], axis=1)

    market = dow_h.merge(nas_h,left_index=True, right_index=True, how="left")
    
    
    tick = yf.Ticker(ticker)
    hist = tick.history(start=starter, end=ender)
    hist = pd.DataFrame(hist)
    
    combined = hist.merge(market, left_index=True, right_index=True, how="left") 

    d = pd.DataFrame()
    for key_word in kw_list_2:
        geoff = pageviewapi.per_article('en.wikipedia', key_word, start, end,
                                    access='all-access', agent='all-agents', granularity='daily')
        dicty = dict(geoff)
        views = pd.DataFrame(dicty["items"])
        views["timestamp"] = pd.to_datetime((views["timestamp"]), format="%Y%m%d%H")
        views = views.set_index("timestamp")
        page = pd.Series(views["views"])
        d[key_word] = page
        
    combined = combined.merge(d, left_index=True, right_index=True, how="right") 
    
    return combined

In [33]:
# example data
apple = Market_scraper(kw_list_1,"AAPL","20210101","20211231")
apple.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,dow_open,dow_high,dow_low,...,nas_open,nas_high,nas_low,nas_close,nas_vol,Apple,Apple Inc.,IPhone,MacBook,MacOS
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2636,12694,7252,511,24790
2021-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2857,14297,7816,618,27483
2021-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2952,15899,8019,698,28429
2021-01-04,132.533082,132.622413,125.823047,128.453461,143301900.0,0.0,0.0,30627.470703,30674.279297,29881.820312,...,12958.519531,12958.719727,12543.240234,12698.450195,6.546740e+09,3529,17389,8188,691,35098
2021-01-05,127.937286,130.766226,127.480679,130.041611,97664900.0,0.0,0.0,30204.250000,30504.890625,30141.779297,...,12665.650391,12828.269531,12665.650391,12818.959961,6.904420e+09,3365,17277,7997,707,35234


## Variable Calculation, Wiki Data

In [147]:
#example code
kw_list = ["Apple Inc.", "IPhone", "MacBook", "MacOS", "Apple Watch"]
apple_wiki = Market_scraper(kw_list,"AAPL", "20190101", "20220425")
apple_wiki["Wiki_total"] = (apple_wiki["Apple Inc."] + apple_wiki["IPhone"] + apple_wiki["MacBook"] + apple_wiki["MacOS"] + apple_wiki["Apple Watch"])
apple_wiki["Stock_total"] = apple_wiki["Close"]

In [148]:
apple_wiki.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,dow_open,dow_high,dow_low,...,nas_low,nas_close,nas_vol,Apple Inc.,IPhone,MacBook,MacOS,Apple Watch,Wiki_total,Stock_total
timestamp,,,,,,,,,,,,,,,,,,,,,
2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9342,8377,713,3258,1864,23554,NaN
2019-01-02,37.543099,38.502947,37.383124,38.277527,148158800.0,0.0,0.0,23058.609375,23413.470703,22928.589844,...,6506.879883,6665.939941,2.261800e+09,12458,9109,881,4518,2472,29438,38.277527
2019-01-03,34.898666,35.320419,34.418744,34.464798,365248800.0,0.0,0.0,23176.390625,23176.390625,22638.410156,...,6457.129883,6463.500000,2.607290e+09,17971,10870,865,4525,2513,36744,34.464798
2019-01-04,35.031976,36.006367,34.855036,35.936073,234428400.0,0.0,0.0,22894.919922,23518.640625,22894.919922,...,6554.240234,6738.859863,2.579550e+09,16865,9986,855,4828,2368,34902,35.936073
2019-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,13002,9762,849,4120,3189,30922,NaN


In [149]:
def variables_creator(df, variable_list, w=7):
    
    '''
    descrition:
    -----------
    takes dataframe and returns new varibles based on recommmended calcualtions, 
    it should be done seporately with internet based and stock based dataframes

    Note: When using for aggreated varaibles, for example Wiki_total, the sum of all the wiki pages daily page view counts, 
    you MUST calculate Wiki_total in the dataframe seperately BEFORE this function can be used. 

    For example if you had wiki page counts for Ford Bronco and Ford Ranger, Wiki_total would equal Ford Bronco + Ford ranger counts, 
    AGAIN Wiki_total must be calculated in the desired dateframe before using this function 

    input:
    ------
    df: dataframe containing the google trends, yahoo finance, and or wikipedia page count data

    variable_list: list of strings to be added to the equations to calculate the new varaibles. 
    ex. insertting the string "Wiki" will add to df[f"{}_total"] to become "Wiki_total"

    w: the window length for one period shift. Default is 7 providing 7 day moving averages for wiki and google data, 
        FOR STOCK DATA THIS WILL NEED TO BE CHANGED TO 5.

    output:
    -------
    df: the same dataframe as was inputted but now containing variables for 
    Momemtum, Disparity, Moving Average, Exponential Moving Aerage, Rator Change, and RSI index score.
    Also containg are moving variables, which are boolean with 1 indicating an increase in the above variables
    '''   
    
    for i in variable_list:
        # Momentum_1
        df[f"{i}_Moment_1"] =  (df[f"{i}_total"] / df[f"{i}_total"].shift(w)) * 100
        # Momentum_2
        df[f"{i}_Moment_2"] =  (df[f"{i}_total"] - df[f"{i}_total"].shift(w)) * 100
        # Momentum_1_s three day shift (instead of w)
        df[f"{i}_Moment_1_s"] =  (df[f"{i}_total"] / df[f"{i}_total"].shift(3)) * 100
        # Momentum_2_s
        df[f"{i}_Moment_2_s"] =  (df[f"{i}_total"] - df[f"{i}_total"].shift(3)) * 100
        # Moving average
        df[f"{i}_MAvg"] = df[f"{i}_total"].rolling(f"{w}d").mean()
        # Moving average 3 day
        df[f"{i}_MAvg_s"] = df[f"{i}_total"].rolling("3d").mean()
        # Disparity
        df[f"{i}_Disparity"] = (df[f"{i}_total"]/df[f"{i}_MAvg"]) * 100
        # Disparity 3 day
        df[f"{i}_Disparity_s"] = (df[f"{i}_total"]/df[f"{i}_MAvg_s"]) * 100
        # Rate of Change Normal Way
        df[f"{i}_ROC"] = (df[f"{i}_total"]-df[f"{i}_total"].shift(w))/(df[f"{i}_total"].shift(w)) *100
        df[f"{i}_ROC_s"] = (df[f"{i}_total"]-df[f"{i}_total"].shift(3))/(df[f"{i}_total"].shift(3)) *100
        #Rate of Change Paper Way (doesn't make sense but just in case)
        df[f'{i}_Rocp'] = (df[f"{i}_total"]/df[f"{i}_Moment_2"]) *100
        # Exponential Moving Average
        df[f"{i}_EMA"] = (df[f"{i}_total"]-df[f"{i}_MAvg"].shift(1))*(2/(w+1))+df[f"{i}_MAvg"].shift(1)

        # calculating the Relative Strength Index, based on 14 day window
        df[f"{i}_diff"] = df[f"{i}_total"].diff(1)
        df[f"{i}_gain"] = df[f"{i}_diff"].clip(lower=0).round(2) #keeps all values above or below a given threshold, lower=lower bound
        df[f"{i}_loss"] = df[f"{i}_diff"].clip(upper=0).round(2)
        df[f'{i}_avg_gain'] = df[f'{i}_gain'].rolling(14).mean()
        df[f'{i}_avg_loss'] = df[f'{i}_loss'].rolling(14).mean()
        df[f'{i}_rs'] = df[f'{i}_avg_gain'] / df[f'{i}_avg_loss']
        df[f'{i}_RSI'] = 100 - (100 / (1.0 + df[f'{i}_rs']))

        # Calculatiing the Move Variables 
        df[f"{i}_Move"] = df[f"{i}_total"] > df[f"{i}_total"].shift(1) 
        df[f"{i}_Move"] = df[f"{i}_Move"].replace({True:1,False: 0})
    
      
        df[f"{i}_MAvg_Move"] = df[f"{i}_MAvg"] > df[f"{i}_MAvg"].shift(1) 
        df[f"{i}_MAvg_Move"] = df[f"{i}_MAvg_Move"].replace({True:1,False: 0})
        df[f"{i}_MAvg_s_Move"] = df[f"{i}_MAvg_s"] > df[f"{i}_MAvg_s"].shift(1) 
        df[f"{i}_MAvg_s_Move"] = df[f"{i}_MAvg_s_Move"].replace({True:1,False: 0})

        df[f"{i}_EMA_Move"] = df[f"{i}_EMA"] > df[f"{i}_EMA"].shift(1) 
        df[f"{i}_EMA_Move"] = df[f"{i}_EMA_Move"].replace({True:1,False: 0})

        df[f"{i}_Disparity_Move"] = df[f"{i}_Disparity"] > df[f"{i}_Disparity"].shift(1) 
        df[f"{i}_Disparity_Move"] = df[f"{i}_Disparity_Move"].replace({True:1,False: 0})
        df[f"{i}_Disparity_s_Move"] = df[f"{i}_Disparity_s"] > df[f"{i}_Disparity_s"].shift(1) 
        df[f"{i}_Disparity_s_Move"] = df[f"{i}_Disparity_s_Move"].replace({True:1,False: 0})

        df[f"{i}_RSI_Move"] = df[f"{i}_RSI"] > df[f"{i}_RSI"].shift(1) 
        df[f"{i}_RSI_Move"] = df[f"{i}_RSI_Move"].replace({True:1,False: 0})
        
    return df

In [153]:
def target_creator(df):
    '''
    description: creates the differnt types of target variables based on tomorrow minus today,
    '''
        
    # target 1, Open(t+1) - Close(t)
    df["target_1"] = (df["Open"].shift(-1) - df["Close"]) > 0
    df["target_1"] = df["target_1"].replace({True:1,False: 0})
    # target 2
    df["target_2"] = (df["Open"].shift(-1) - df["Open"]) > 0
    df["target_2"] = df["target_2"].replace({True:1,False: 0})
    # target 3
    df["target_3"] = (df["Close"].shift(-1) - df["Close"]) > 0
    df["target_3"] = df["target_3"].replace({True:1,False: 0})
    # target 4
    df["target_4"] = (df["Close"].shift(-1) - df["Open"]) > 0
    df["target_4"] = df["target_4"].replace({True:1,False: 0})
    # target 5
    df["target_5"] = (df["Volume"].shift(-1) - df["Volume"]) > 0
    df["target_5"] = df["target_5"].replace({True:1,False: 0})
 
    return df

In [155]:
target_creator(apple_wiki)[["Wiki_total","Wiki_MAvg", 'Stock_total', 'Stock_MAvg']]

,Wiki_total,Wiki_MAvg,Stock_total,Stock_MAvg
timestamp,,,,
2019-01-01,23554,23554.000000,NaN,NaN
2019-01-02,29438,26496.000000,38.277527,38.277527
2019-01-03,36744,29912.000000,34.464798,36.371162
2019-01-04,34902,31159.500000,35.936073,36.226133
2019-01-05,30922,31112.000000,NaN,36.226133
...,...,...,...,...
2022-04-21,56350,56186.857143,166.419998,166.529999
2022-04-22,52851,55493.714286,161.789993,165.581998
2022-04-23,56975,55623.714286,NaN,165.581998


In [159]:
apple_wiki.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,dow_open,dow_high,dow_low,...,Stock_MAvg_s_Move,Stock_EMA_Move,Stock_Disparity_Move,Stock_Disparity_s_Move,Stock_RSI_Move,target_1,target_2,target_3,target_4,target_5
timestamp,,,,,,,,,,,,,,,,,,,,,
2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2019-01-02,37.543099,38.502947,37.383124,38.277527,148158800.0,0.0,0.0,23058.609375,23413.470703,22928.589844,...,0,0,0,0,0,0,0,0,0,1
2019-01-03,34.898666,35.320419,34.418744,34.464798,365248800.0,0.0,0.0,23176.390625,23176.390625,22638.410156,...,0,0,0,0,0,1,1,1,1,0
2019-01-04,35.031976,36.006367,34.855036,35.936073,234428400.0,0.0,0.0,22894.919922,23518.640625,22894.919922,...,0,0,1,1,0,0,0,0,0,0
2019-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
